In [17]:
# !unzip ../input/dogs-vs-cats/test1.zip
# !unzip ../input/dogs-vs-cats/train.zip

In [26]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

In [38]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense

from sklearn.model_selection import train_test_split


In [39]:
image_size = (128, 128)
channels = (3,)

image_dims = image_size + channels


In [56]:
import cv2
from PIL import Image

train_dir = './train/'
test_dir = './test1/'
filenames=os.listdir("./train/")

image_size = (128, 128)
channels = (3,)

image_dims = image_size + channels

categories=[]
for img in filenames:
    category=img.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({'filename': filenames, 'category': categories})

In [72]:
df["category"] = df["category"].replace({0:'cat',1:'dog'})
train_df,validate_df = train_test_split(df,
                                        test_size=0.20,
                                        random_state=32)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=32

In [73]:
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',
                                            patience = 2,
                                            verbose = 1,
                                            factor = 0.5,
                                            min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [84]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                   rescale=1./255,
                                   shear_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1
                                  )
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    './train',
                                                    x_col='filename',
                                                    y_col='category',
                                                    target_size=image_size,
                                                    class_mode='categorical',
                                                    batch_size=batch_size,)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              './train',                                                                
                                                              x_col='filename',
                                                              y_col='category',
                                                              target_size=image_size,
                                                              class_mode='categorical',
                                                              batch_size=batch_size)
test_datagen = ImageDataGenerator(rotation_range=15,
                                  rescale=1./255,
                                  shear_range=0.1,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1)
test_generator = train_datagen.flow_from_dataframe(train_df,
                                                   './test1',
                                                   x_col='filename',
                                                   y_col='category',
                                                   target_size= image_size,
                                                   class_mode='categorical',
                                                   batch_size=batch_size,
                                                  shuffle=False)

In [77]:
model = Sequential()
def predict(): 
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=image_dims))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3),activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(2))
    model.add(Activation('sigmoid'))
    
    return model

In [78]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',             
              metrics=['accuracy'])
model = predict()

In [79]:
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 batch_normalization_6 (Batc  (None, 126, 126, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 63, 63, 32)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_7 (Batc  (None, 61, 61, 64)      

In [85]:
epochs=30
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=10
    callbacks = callbacks
    )


In [86]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

_, acc = model.evaluate(test_generator, validate_df)
print("Accuracy = ", (acc * 100.0), "%")